In [14]:
import re
import pandas as pd

from glob import glob

In [15]:
def unity(unit: str):
    if unit[-2:] == "us":
        unit = float(unit[:-2])
    elif unit[-2:] == "ms":
        unit = float(unit[:-2]) * 10 ** 3
    elif unit[-1] == "s":
        unit = float(unit[:-1]) * 10 ** 6
    return unit

In [16]:
logs = glob("./logs/bigscience/bloom-1b1/**/*.log", recursive=True)

for log in logs:
    with open(log, "r") as f:
        lines = f.readlines()
    for idx, line in enumerate(lines):
        if "Result of profile" in line:
            lines = lines[idx+1:]
            break

In [24]:
pattern = r"\s{2,}"
columns = re.split(pattern, lines[1].strip())
content = []

for line in lines[3:-4]:
    try:
        splited = re.split(pattern, line.strip())
        temp_dict = {columns[i]:splited[i] for i in range(len(columns))}
        temp_dict["Self CPU"] = unity(temp_dict["Self CPU"])
        temp_dict["CPU total"] = unity(temp_dict["CPU total"])
        temp_dict["Self CUDA"] = unity(temp_dict["Self CUDA"])
        temp_dict["CUDA total"] = unity(temp_dict["CUDA total"])
        content.append(temp_dict)
    except Exception as e:
        print(e)
        print(splited)
        break

In [25]:
df = pd.DataFrame(content)
df

,Name,Self CPU %,Self CPU,CPU total %,CPU total,CPU time avg,Self CUDA,Self CUDA %,CUDA total,CUDA time avg,# of Calls
0,cudaLaunchKernel,55.00%,6903000.0,55.00%,6904000.0,150.691us,573798.0,5.41%,585342.0,12.776us,45815
1,aten::addmm,16.50%,2070000.0,17.13%,2150000.0,1.400ms,1918000.0,18.10%,2021000.0,1.316ms,1536
2,cudaMemcpyAsync,13.76%,1727000.0,13.76%,1727000.0,349.825us,26538.0,0.25%,26538.0,5.374us,4938
3,cudaLaunchKernelExC,2.52%,316827.0,2.52%,316839.0,68.417us,201649.0,1.90%,201649.0,43.543us,4631
4,cudaMalloc,2.43%,304491.0,2.43%,304491.0,2.559ms,1998.0,0.02%,2108.0,17.714us,119
...,...,...,...,...,...,...,...,...,...,...,...
235,autograd::engine::evaluate_function: SelectBac...,-0.11%,-13619.0,3.47%,435423.0,431.967us,0.0,0.00%,125018.0,124.026us,1008
236,autograd::engine::evaluate_function: SoftmaxBa...,-0.12%,-15571.0,2.21%,277003.0,824.414us,0.0,0.00%,27566.0,82.042us,336
237,aten::slice_backward,-0.34%,-42080.0,9.31%,1168000.0,1.128ms,0.0,0.00%,194357.0,187.603us,1036
238,autograd::engine::evaluate_function: SliceBack...,-0.43%,-54591.0,9.34%,1172000.0,1.131ms,0.0,0.00%,195719.0,188.918us,1036


In [32]:
df.sort_values(by="Self CUDA", ascending=False)[:10]

,Name,Self CPU %,Self CPU,CPU total %,CPU total,CPU time avg,Self CUDA,Self CUDA %,CUDA total,CUDA time avg,# of Calls
15,aten::mm,0.25%,31784.0,3.85%,483735.0,177.063us,5832000.0,55.03%,5987000.0,2.192ms,2732
174,sm80_xmma_gemm_f32f32_f32f32_f32_tn_n_tilesize...,0.00%,0.0,0.00%,0.0,0.000us,2044000.0,19.28%,2044000.0,1.825ms,1120
1,aten::addmm,16.50%,2070000.0,17.13%,2150000.0,1.400ms,1918000.0,18.10%,2021000.0,1.316ms,1536
194,void cutlass::Kernel<cutlass_80_simt_sgemm_128...,0.00%,0.0,0.00%,0.0,0.000us,1883000.0,17.77%,1883000.0,1.843ms,1022
198,void cutlass::Kernel<cutlass_80_simt_sgemm_256...,0.00%,0.0,0.00%,0.0,0.000us,1491000.0,14.07%,1491000.0,1.479ms,1008
179,sm80_xmma_gemm_f32f32_f32f32_f32_tn_n_tilesize...,0.00%,0.0,0.00%,0.0,0.000us,852231.0,8.04%,852231.0,1.110ms,768
195,sm80_xmma_gemm_f32f32_f32f32_f32_nn_n_tilesize...,0.00%,0.0,0.00%,0.0,0.000us,799467.0,7.54%,799467.0,57.105ms,14
12,aten::mul,0.44%,54932.0,13.82%,1735000.0,150.883us,614168.0,5.79%,729449.0,63.441us,11498
0,cudaLaunchKernel,55.00%,6903000.0,55.00%,6904000.0,150.691us,573798.0,5.41%,585342.0,12.776us,45815
199,void cutlass::Kernel<cutlass_80_simt_sgemm_256...,0.00%,0.0,0.00%,0.0,0.000us,530783.0,5.01%,530783.0,1.580ms,336


In [35]:
df.sort_values(by="Self CUDA", ascending=False)[:10]["Self CUDA"].sum() / 10 ** 6

16.538447

In [28]:
print(df["Self CPU"].sum() / 10 ** 6)
print(df["CPU total"].sum() / 10 ** 6)
print(df["Self CUDA"].sum() / 10 ** 6)
print(df["CUDA total"].sum() / 10 ** 6)

12.550633
46.811438
22.270618
47.439649
